Transform Parquet Files from Azure Data Lake (Bronze to Silver)

In [0]:
# Set up Spark and define paths
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date
from pyspark.sql.functions import to_timestamp, year, month
from pyspark.sql.functions import col, year, month, countDistinct, avg
from pyspark.sql.functions import year, month, dayofmonth

In [0]:
spark.conf.set(
    f"fs.azure.sas.mainstack.mainstackstorage.blob.core.windows.net",
    "sv=2024-11-04&ss=bfqt&srt=sco&sp=rwdlacux&se=2025-06-06T08:28:36Z&st=2025-05-26T00:28:36Z&spr=https&sig=8cVvbdaAwObIuwUV%2BYysr1memI4NsA1UqI0C4sKU%2FgE%3D"
)

In [0]:
# Mounting Demographics
df_demographics = spark.read.parquet(
    "wasbs://mainstack@mainstackstorage.blob.core.windows.net/bronze/demographics/demographics.parquet"
)

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
File <command-7756651328113389>, line 2
      1 # Mounting Demographics
----> 2 df_demographics = spark.read.parquet(
      3     "wasbs://mainstack@mainstackstorage.blob.core.windows.net/bronze/demographics/demographics.parquet"
      4 )

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:544, in DataFrameReader.parquet(self, *paths, **options)
    533 int96RebaseMode = options.get("int96RebaseMode", None)
    534 self._set_opts(
    535     mergeSchema=mergeSchem

In [0]:
# Show schema and sample data
df_demographics.printSchema()
df_demographics.show(5)

root
 |-- user_id: string (nullable = true)
 |-- age: long (nullable = true)
 |-- age_group: string (nullable = true)
 |-- device: string (nullable = true)
 |-- creator_segment: string (nullable = true)
 |-- signup_date: string (nullable = true)
 |-- active_status: string (nullable = true)
 |-- subscription_tier: string (nullable = true)

+---------------+---+---------+----------+----------------+--------------------+-------------+-----------------+
|        user_id|age|age_group|    device| creator_segment|         signup_date|active_status|subscription_tier|
+---------------+---+---------+----------+----------------+--------------------+-------------+-----------------+
|9MAw8EMThcoVgAg| 51|      50s|Mobile iOS|Music Production|2025-04-09T15:24:...|       Active|             Free|
|5s61lqRQTww0c65| 18|      10s|    Tablet|       Education|2025-02-12T02:55:...|       Active|             Free|
|GrpcDwIB4dy4aPz| 64|      60s|   Desktop|     Digital Art|2023-11-20T18:49:...|     Inactive|

In [0]:
# Mounting Competitors
df_competitors = spark.read.format("parquet").load(
    "wasbs://mainstack@mainstackstorage.blob.core.windows.net/bronze/competitors/competitors.parquet"
)
df_competitors.show()


+---------------+---------------+-------------+--------------------+---------------------+-------------------+-----------------------+---------------+----------------+------------------+
|  competitor_id|  platform_name|founding_year|headquarter_location|commission_percentage|supported_countries|market_share_percentage|sentiment_score|app_store_rating|google_play_rating|
+---------------+---------------+-------------+--------------------+---------------------+-------------------+-----------------------+---------------+----------------+------------------+
|qbkStHDYF4Ztwjh|        Patreon|         2020|Reneeborough, Ger...|                    7|                 20|                  29.65|            4.1|             3.5|               4.2|
|jCoZnYTsdEZ1aJJ|          Ko-fi|         2017|Kennethville, Canada|                   15|                 74|                  29.09|            4.2|             4.6|               4.8|
|a5sdfzbWXVYbVFK|        Gumroad|         2011|   Melaniemouth, U

In [0]:
# Mounting Content
df_content = spark.read.parquet(
    "wasbs://mainstack@mainstackstorage.blob.core.windows.net/bronze/content/content.parquet"
)
df_content.show()



+---------------+---------------+---------+------------+--------------------+--------------------+--------------------+-----------+-----------+-----+--------+------+-----+-----------+-----------------+---------------+----------------+--------------------+-----------------+
|     content_id|     creator_id| platform|content_type|               title|         description|      published_date|day_of_week|time_of_day|likes|comments|shares|views|impressions|average_view_time|completion_rate| creator_segment|monetization_enabled|revenue_generated|
+---------------+---------------+---------+------------+--------------------+--------------------+--------------------+-----------+-----------+-----+--------+------+-----+-----------+-----------------+---------------+----------------+--------------------+-----------------+
|Dov1vvIZ8gParAX|cyzh8h7IhKFzKaN|  Twitter|       Image|Total full-range ...|Form move so scor...|2025-05-22T07:50:...|   Thursday|      07:50| 4240|     295|   209|16926|      3

In [0]:
# Mounting engagement
df_engagement = spark.read.parquet(
    "wasbs://mainstack@mainstackstorage.blob.core.windows.net/bronze/engagement/engagement.parquet"
)

df_content.show()

+---------------+---------------+---------+------------+--------------------+--------------------+--------------------+-----------+-----------+-----+--------+------+-----+-----------+-----------------+---------------+----------------+--------------------+-----------------+
|     content_id|     creator_id| platform|content_type|               title|         description|      published_date|day_of_week|time_of_day|likes|comments|shares|views|impressions|average_view_time|completion_rate| creator_segment|monetization_enabled|revenue_generated|
+---------------+---------------+---------+------------+--------------------+--------------------+--------------------+-----------+-----------+-----+--------+------+-----+-----------+-----------------+---------------+----------------+--------------------+-----------------+
|Dov1vvIZ8gParAX|cyzh8h7IhKFzKaN|  Twitter|       Image|Total full-range ...|Form move so scor...|2025-05-22T07:50:...|   Thursday|      07:50| 4240|     295|   209|16926|      3

In [0]:
# Clean/Transform demographic table
df_demographics_clean = (
    df_demographics
    .withColumn("signup_date", to_timestamp("signup_date"))
    .withColumn("signup_year", year("signup_date"))
    .withColumn("signup_month", month("signup_date"))
)

df_demographics_clean.write.mode("overwrite").parquet("/mnt/silver/demographics")


In [0]:
# Clean/Transform demographic table
df_competitors_clean = (
    df_competitors
    .withColumn("commission_percentage", col("commission_percentage").cast("double"))
    .withColumn("market_share_percentage", col("market_share_percentage").cast("double"))
    .withColumn("sentiment_score", col("sentiment_score").cast("double"))
)

df_competitors_clean.write.mode("overwrite").parquet("/mnt/silver/competitors")


In [0]:
# Clean/Transform content table

df_content_clean = (
    df_content
    .withColumn("published_date", to_timestamp("published_date"))
    .withColumn("monetization_enabled", col("monetization_enabled").cast("boolean"))
)

df_content_clean.write.mode("overwrite").parquet("/mnt/silver/content")


In [0]:
# Clean/Transform engagement table
df_engagement_clean = df_engagement.select(
    "creator_id",
    "platform",
    "date",
    "total_likes",
    "total_comments",
    "total_shares",
    "average_view_duration",
    "impressions",
    "reach",
    "click_through_rate"
)

df_engagement_clean.write.mode("overwrite").parquet("/mnt/silver/engagement")
df_engagement_clean.printSchema()




root
 |-- creator_id: string (nullable = true)
 |-- platform: string (nullable = true)
 |-- date: string (nullable = true)
 |-- total_likes: long (nullable = true)
 |-- total_comments: long (nullable = true)
 |-- total_shares: long (nullable = true)
 |-- average_view_duration: double (nullable = true)
 |-- impressions: long (nullable = true)
 |-- reach: long (nullable = true)
 |-- click_through_rate: double (nullable = true)



In [0]:
# Confirm mounting of silver table

display(dbutils.fs.ls("/mnt/silver"))

path,name,size,modificationTime
dbfs:/mnt/silver/competitors/,competitors/,0,0
dbfs:/mnt/silver/content/,content/,0,0
dbfs:/mnt/silver/demographics/,demographics/,0,0
dbfs:/mnt/silver/dim_creators/,dim_creators/,0,0
dbfs:/mnt/silver/dim_platforms/,dim_platforms/,0,0
dbfs:/mnt/silver/dim_time/,dim_time/,0,0
dbfs:/mnt/silver/engagement/,engagement/,0,0
dbfs:/mnt/silver/fact_content/,fact_content/,0,0
dbfs:/mnt/silver/fact_engagement/,fact_engagement/,0,0


In [0]:
sas_token ="sv=2024-11-04&ss=bfqt&srt=sco&sp=rwdlacux&se=2025-06-06T08:28:36Z&st=2025-05-26T00:28:36Z&spr=https&sig=8cVvbdaAwObIuwUV%2BYysr1memI4NsA1UqI0C4sKU%2FgE%3D"  
storage_account = "mainstackstorage"
container = "mainstack"

dbutils.fs.mount(
  source = f"wasbs://{container}@{storage_account}.blob.core.windows.net/silver",
  mount_point = "/mnt/silver",
  extra_configs = {
    f"fs.azure.sas.{container}.{storage_account}.blob.core.windows.net": sas_token
  }
)


---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
File <command-8641583434970591>, line 5
      2 storage_account = "mainstackstorage"
      3 container = "mainstack"
----> 5 dbutils.fs.mount(
      6   source = f"wasbs://{container}@{storage_account}.blob.core.windows.net/silver",
      7   mount_point = "/mnt/silver",
      8   extra_configs = {
      9     f"fs.azure.sas.{container}.{storage_account}.blob.core.windows.net": sas_token
     10   }
     11 )

File /databricks/python_shell/dbruntime/dbutils.py:364, in DBUtils.FSHandler.prettify_exception_message.<locals>.f_with_exception_handling(*args, **kwargs)
    362 exc.__context__ = None
    363 exc.__cause__ = None
--> 364 raise exc

ExecutionError: An error occurred while calling o399.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/silver; nested exceptio

Transform Parquet Files from Azure Data Lake (Silver to Gold)

In [0]:
# Create dim_creator table

df_dim_creators = (
    df_demographics_clean
    .select("user_id", "age", "age_group", "device", "creator_segment", "subscription_tier")
    .dropDuplicates(["user_id"])
    .withColumnRenamed("user_id", "creator_id")
)

df_dim_creators.write.mode("overwrite").parquet("/mnt/gold/dim_creators")

df_dim_creators.printSchema()


root
 |-- creator_id: string (nullable = true)
 |-- age: long (nullable = true)
 |-- age_group: string (nullable = true)
 |-- device: string (nullable = true)
 |-- creator_segment: string (nullable = true)
 |-- subscription_tier: string (nullable = true)



In [0]:
# Dimension: Platforms (from content)

df_dim_platforms = (
    df_content_clean
    .select("platform")
    .dropDuplicates()
)

df_dim_platforms.write.mode("overwrite").parquet("/mnt/gold/dim_platforms")

df_dim_platforms.printSchema()

root
 |-- platform: string (nullable = true)



In [0]:
# Fact: Engagement Metrics

df_fact_engagement = (
    df_engagement_clean
    .withColumn("engagement_score", col("total_likes") + col("total_comments") + col("total_shares"))
    .withColumn("engagement_rate", (col("total_likes") + col("total_comments") + col("total_shares")) / col("impressions"))
    .withColumn("avg_watch_time", col("average_view_duration"))
    .select(
        "creator_id", "platform", "date", "total_likes", "total_comments", "total_shares",
        "impressions", "reach", "engagement_score", "engagement_rate", "avg_watch_time", "click_through_rate"
    )
)

df_fact_engagement.write.mode("overwrite").parquet("/mnt/gold/fact_engagement")

df_fact_engagement.printSchema()

root
 |-- creator_id: string (nullable = true)
 |-- platform: string (nullable = true)
 |-- date: string (nullable = true)
 |-- total_likes: long (nullable = true)
 |-- total_comments: long (nullable = true)
 |-- total_shares: long (nullable = true)
 |-- impressions: long (nullable = true)
 |-- reach: long (nullable = true)
 |-- engagement_score: long (nullable = true)
 |-- engagement_rate: double (nullable = true)
 |-- avg_watch_time: double (nullable = true)
 |-- click_through_rate: double (nullable = true)



In [0]:
# Fact: Content Performance

df_fact_content = (
    df_content_clean
    .select("content_id", "creator_id", "platform", "content_type", "likes", "comments", "view" "published_date", "monetization_enabled", "revenue_generated")
)

df_fact_content.write.mode("overwrite").parquet("/mnt/gold/fact_content")

df_fact_content.printSchema()

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-4768733374406753>, line 4
      1 # Fact: Content Performance
      3 df_fact_content = (
----> 4     df_content_clean
      5     .select("content_id", "creator_id", "platform", "content_type", "likes", "comments", "view" "published_date", "monetization_enabled", "revenue_generated")
      6 )
      8 df_fact_content.write.mode("overwrite").parquet("/mnt/gold/fact_content")
     10 df_fact_content.printSchema()

NameError: name 'df_content_clean' is not defined

In [0]:

df_dim_time = (
    df_content_clean
    .select("published_date")
    .withColumn("year", year("published_date"))
    .withColumn("month", month("published_date"))
    .withColumn("day", dayofmonth("published_date"))  # Now works!
    .dropDuplicates()
)

df_dim_time.write.mode("overwrite").parquet("/mnt/gold/dim_time")

df_dim_time.printSchema()

print("✅ Gold layer models saved successfully")

root
 |-- published_date: timestamp (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)

✅ Gold layer models saved successfully


In [0]:
from pyspark.sql.functions import col, to_timestamp, year, month, dayofmonth

def cast_column(df, column, target_type):
    return df.withColumn(column, col(column).cast(target_type))

def parse_timestamp(df, column):
    return df.withColumn(column, to_timestamp(column))

def add_date_parts(df, timestamp_col):
    return (df
        .withColumn("year", year(timestamp_col))
        .withColumn("month", month(timestamp_col))
        .withColumn("day", dayofmonth(timestamp_col)))

def deduplicate(df, subset_cols):
    return df.dropDuplicates(subset=subset_cols)

def clean_column_names(df):
    for col_name in df.columns:
        df = df.withColumnRenamed(col_name, col_name.strip().lower().replace(" ", "_"))
    return df

df_dim_time.write.mode("overwrite").parquet("/mnt/gold/dim_time")

df_dim_time.printSchema()

print("✅ Gold layer models saved successfully")

root
 |-- published_date: timestamp (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)

✅ Gold layer models saved successfully


In [0]:
display(dbutils.fs.ls("/mnt/gold"))

path,name,size,modificationTime
dbfs:/mnt/gold/dim_creators/,dim_creators/,0,0
dbfs:/mnt/gold/dim_platforms/,dim_platforms/,0,0
dbfs:/mnt/gold/dim_time/,dim_time/,0,0
dbfs:/mnt/gold/fact_content/,fact_content/,0,0
dbfs:/mnt/gold/fact_engagement/,fact_engagement/,0,0


In [0]:
df_dim_time.count()

1000

In [0]:
sas_token ="sv=2024-11-04&ss=bfqt&srt=sco&sp=rwdlacux&se=2025-06-06T08:28:36Z&st=2025-05-26T00:28:36Z&spr=https&sig=8cVvbdaAwObIuwUV%2BYysr1memI4NsA1UqI0C4sKU%2FgE%3D"  
storage_account = "mainstackstorage"
container = "mainstack"

dbutils.fs.mount(
  source = f"wasbs://{container}@{storage_account}.blob.core.windows.net/gold",
  mount_point = "/mnt/gold",
  extra_configs = {
    f"fs.azure.sas.{container}.{storage_account}.blob.core.windows.net": sas_token
  }
)


---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
File <command-4886740239424571>, line 5
      2 storage_account = "mainstackstorage"
      3 container = "mainstack"
----> 5 dbutils.fs.mount(
      6   source = f"wasbs://{container}@{storage_account}.blob.core.windows.net/gold",
      7   mount_point = "/mnt/gold",
      8   extra_configs = {
      9     f"fs.azure.sas.{container}.{storage_account}.blob.core.windows.net": sas_token
     10   }
     11 )

File /databricks/python_shell/dbruntime/dbutils.py:364, in DBUtils.FSHandler.prettify_exception_message.<locals>.f_with_exception_handling(*args, **kwargs)
    362 exc.__context__ = None
    363 exc.__cause__ = None
--> 364 raise exc

ExecutionError: An error occurred while calling o399.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/gold; nested exception is: 

In [0]:
dbutils.fs.unmount("/mnt/gold")

/mnt/gold has been unmounted.


True

Moving Gold Folder to SQL Database

In [0]:
# Reload Gold tables from /mnt/gold
df_dim_creators = spark.read.parquet("/mnt/gold/dim_creators")
df_dim_platforms = spark.read.parquet("/mnt/gold/dim_platforms")
df_fact_engagement = spark.read.parquet("/mnt/gold/fact_engagement")
df_fact_content = spark.read.parquet("/mnt/gold/fact_content")
df_dim_time = spark.read.parquet("/mnt/gold/dim_time")


In [0]:
jdbc_url = "jdbc:postgresql://mainstackdb.postgres.database.azure.com:5432/mainstack_analytics"
connection_properties = {
    "user": "yusuf",
    "password": "School1.",
    "driver": "org.postgresql.Driver",
    "sslmode": "require"
}

# Write dim_creators
df_dim_creators.write.jdbc(
    url=jdbc_url,
    table="dim_creators",
    mode="overwrite",
    properties=connection_properties
)

# Write dim_platforms
df_dim_platforms.write.jdbc(
    url=jdbc_url,
    table="dim_platforms",
    mode="overwrite",
    properties=connection_properties
)

# Write fact_engagement
df_fact_engagement.write.jdbc(
    url=jdbc_url,
    table="fact_engagement",
    mode="overwrite",
    properties=connection_properties
)

# Write fact_content
df_fact_content.write.jdbc(
    url=jdbc_url,
    table="fact_content",
    mode="overwrite",
    properties=connection_properties
)

# Write dim_time
df_dim_time.write.jdbc(
    url=jdbc_url,
    table="dim_time",
    mode="overwrite",
    properties=connection_properties
)

print("✅ All gold-layer tables exported to PostgreSQL successfully.")


✅ All gold-layer tables exported to PostgreSQL successfully.


In [0]:
%sql
SELECT * FROM df_dim_creators LIMIT 5;

com.databricks.backend.common.rpc.SparkDriverExceptions$SQLExecutionException: org.apache.spark.sql.AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `df_dim_creators` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 1 pos 14;
'GlobalLimit 5
+- 'LocalLimit 5
 +- 'Project [*]
 +- 'UnresolvedRelation [df_dim_creators], [], false

	at org.apache.spark.sql.catalyst.analysis.package$AnalysisErrorAt.tableNotFound(package.scala:90)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.$anonfun$checkAnalysis0$2(CheckAnalysis.scala:218)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.$anonfun$checkAnalysis0$2$adapted(CheckAnalysis.scala:191)
	at org.apache.spark.sql.catalyst.trees.TreeNode.foreachUp(TreeNode.scala:248)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1(TreeNode.scala:247)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1$adapted(TreeNode.scala:247)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection.Iterator.foreach$(Iterator.scala:943)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1431)
	at scala.collection.IterableLike.foreach(IterableLike.scala:74)
	at scala.collection.IterableLike.foreach$(IterableLike.scala:73)
	at scala.collection.AbstractIterable.foreach(Iterable.scala:56)
	at org.apache.spark.sql.catalyst.trees.TreeNode.foreachUp(TreeNode.scala:247)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1(TreeNode.scala:247)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1$adapted(TreeNode.scala:247)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection.Iterator.foreach$(Iterator.scala:943)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1431)
	at scala.collection.IterableLike.foreach(IterableLike.scala:74)
	at scala.collection.IterableLike.foreach$(IterableLike.scala:73)
	at scala.collection.AbstractIterable.foreach(Iterable.scala:56)
	at org.apache.spark.sql.catalyst.trees.TreeNode.foreachUp(TreeNode.scala:247)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1(TreeNode.scala:247)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1$adapted(TreeNode.scala:247)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection.Iterator.foreach$(Iterator.scala:943)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1431)
	at scala.collection.IterableLike.foreach(IterableLike.scala:74)
	at scala.collection.IterableLike.foreach$(IterableLike.scala:73)
	at scala.collection.AbstractIterable.foreach(Iterable.scala:56)
	at org.apache.spark.sql.catalyst.trees.TreeNode.foreachUp(TreeNode.scala:247)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.checkAnalysis0(CheckAnalysis.scala:191)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.checkAnalysis0$(CheckAnalysis.scala:173)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.checkAnalysis0(Analyzer.scala:363)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.$anonfun$checkAnalysis$1(CheckAnalysis.scala:169)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.spark.util.FrameProfiler$.record(FrameProfiler.scala:94)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.checkAnalysis(CheckAnalysis.scala:159)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.checkAnalysis$(CheckAnalysis.scala:159)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.checkAnalysis(Analyzer.scala:363)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.$anonfun$executeAndCheck$2(Analyzer.scala:418)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.sql.catalyst.QueryPlanningTracker$.withTrack